In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchvision.transforms as transformers
from torchvision.datasets import ImageFolder
from tqdm.notebook import tqdm
import sklearn
from sklearn.preprocessing import normalize
from tqdm import tqdm

import glob
import time

import math
import os

In [2]:
initial_states = []

path = "../input_data/"

for dir, sub_dir, files in os.walk(path):
    for file in sorted(files):
        #print(file)
        temp = pd.read_csv((path+file),index_col=None, header=0)
        initial_states.append(temp)

initial_states_df = pd.concat(initial_states,axis=0,ignore_index=True)

hold = initial_states_df
x = initial_states_df.iloc[:,2:].values
x = normalize(x,norm='l2')
hold = pd.concat([hold['File ID'],pd.DataFrame(x)],axis=1)
initial_states_normalized = hold
#'2000-08-02 04:50:33'
timestamps = initial_states_df['Timestamp']
initial_states_df

,File ID,Timestamp,Semi-major Axis (km),Eccentricity,Inclination (deg),RAAN (deg),Argument of Perigee (deg),True Anomaly (deg),Latitude (deg),Longitude (deg),Altitude (km)
0,0,2000-08-02 04:50:33,6826.387247,0.003882,87.275306,144.135111,257.314389,102.383270,43.637815,-62.543128,466.448890
1,1,2000-08-03 19:51:01,6826.327748,0.003879,87.275694,143.529694,250.438806,109.273118,43.444458,70.139709,463.435053
2,2,2000-08-05 05:40:05,6819.634802,0.004114,87.268611,142.972111,244.549389,115.138737,19.764250,104.521278,471.625453
3,3,2000-08-06 05:02:20,6819.606603,0.004134,87.268194,142.608389,241.172000,118.545161,12.450738,112.239558,470.385914
4,4,2000-08-08 20:54:57,6819.425918,0.004178,87.264611,141.605111,228.779611,130.982981,-8.776992,-130.559634,468.911226
...,...,...,...,...,...,...,...,...,...,...,...
8114,8114,2019-12-25 00:00:00,6765.013678,0.005730,87.863978,102.587920,240.608198,187.758342,69.535173,108.291937,443.930606
8115,8115,2019-12-27 00:00:00,6801.130577,0.002172,90.690901,99.760357,152.602156,226.350702,20.225336,99.507926,435.492910
8116,8116,2019-12-28 00:00:00,6805.864837,0.001925,91.053632,96.918243,43.442569,131.160787,4.265831,-83.003782,436.652863
8117,8117,2019-12-30 00:00:00,6774.300973,0.004785,88.598951,92.814340,320.652681,164.621884,53.716936,-89.084995,441.438958


In [3]:
class FullDataset(Dataset):
    def __init__(self, initial_states_df, timestamps, density_length=432, goes_length=86400, omni2_length=1440, density_dir='../data/dataset/test/sat_density', goes_dir="../data/dataset/test/goes",
                 omni2_dir="../data/dataset/test/omni2"):
        self.data = initial_states_df.reset_index(drop=True)
        self.density_dir = density_dir
        self.goes_dir = goes_dir
        self.omni2_dir = omni2_dir
        self.density_length = density_length
        self.goes_length = goes_length
        self.omni2_length = omni2_length
        self.timestamps = timestamps
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        #timestamp = row['Timestamp']
        static_input = torch.tensor(row.drop('File ID').fillna(0.0).values, dtype=torch.float32)
        
        file_id = str(int(row['File ID'])).zfill(5)
        
        density_file = glob.glob(os.path.join(self.density_dir, f"*-{file_id}-*.csv"))
        goes_file = glob.glob(os.path.join(self.goes_dir, f"*-{file_id}-*.csv"))
        omni2_file = glob.glob(os.path.join(self.omni2_dir, f"*-{file_id}-*.csv"))

        #pos = len(self.density_dir)+1
        #density_sat = density_file[0][pos:pos+6]

        density_df = ((pd.read_csv(density_file[0])))
        density_df['Orbit Mean Density (kg/m^3)'] = np.where(density_df['Orbit Mean Density (kg/m^3)']>=1,np.nan,density_df['Orbit Mean Density (kg/m^3)'])
        if density_df.shape[0] > self.density_length:
            density_df = density_df[:self.density_length]
        elif density_df.shape[0] < self.density_length:
            padding = pd.DataFrame(np.empty((self.density_length-density_df.shape[0],2)),columns=density_df.columns)
            padding[:] = np.nan
            density_df = pd.concat((density_df,padding),ignore_index=True)
        density_df_mask = (pd.notnull(density_df)).astype(int)
        density_tensor = torch.tensor(density_df['Orbit Mean Density (kg/m^3)'].fillna(0.0).values, dtype=torch.float32)
        density_df_mask_tensor = torch.tensor(density_df_mask.iloc[:,1].values, dtype=torch.float32)

        goes_df = pd.read_csv(goes_file[0])
        if goes_df.shape[0] > self.goes_length:
            goes_df = goes_df[goes_df.shape[0]-self.goes_length:goes_df.shape[0]]
        elif goes_df.shape[0] < self.goes_length:
            padding = pd.DataFrame(np.empty((self.goes_length-goes_df.shape[0],43)),columns=goes_df.columns)
            padding[:] = np.nan
            goes_df = pd.concat((padding,goes_df),ignore_index=True)
        goes_mask = (~pd.isnull(goes_df)).astype(int)
        goes_valid_mask = ((goes_df['xrsa_flag'] == 0.0) & (goes_df['xrsb_flag'] == 0.0)).astype(int)
        goes_mask = goes_mask.mul(goes_valid_mask.values,axis=0)
        goes_tensor = torch.tensor(normalize(goes_df.iloc[:, 1:].fillna(0.0).values, norm='l2'), dtype=torch.float32)
        goes_mask_tensor = torch.tensor(goes_mask.iloc[:, 1:].values, dtype=torch.float32)
        
        omni2_df = pd.read_csv(omni2_file[0])
        if omni2_df.shape[0] > self.omni2_length:
            omni2_df = omni2_df[omni2_df.shape[0]-self.omni2_length:omni2_df.shape[0]]
        elif goes_df.shape[0] < self.omni2_length:
            padding = pd.DataFrame(np.empty((self.omni2_length-omni2_df.shape[0],58)),columns=omni2_df.columns)
            padding[:] = np.nan
            omni2_df = pd.concat((padding,omni2_df),ignore_index=True)
        omni2_tensor = torch.tensor(normalize(omni2_df.iloc[:, :57].fillna(0.0).values.astype(float), norm='l2'), dtype=torch.float32)
        omni2_mask = (~pd.isnull(omni2_df)).astype(int)
        omni2_mask_tensor = torch.tensor(omni2_mask.iloc[:, :57].values, dtype=torch.float32) 

        return static_input, density_tensor, density_df_mask_tensor, goes_tensor, goes_mask_tensor, omni2_tensor, omni2_mask_tensor, self.timestamps[idx], idx



In [3]:

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=10000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(-torch.arange(0, d_model, 2) * math.log(10000.0) / d_model)
        pe[:, 0::2] = torch.sin(pos * div_term)
        pe[:, 1::2] = torch.cos(pos * div_term)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :].to(x.device)

class STORMTransformer(nn.Module):
    def __init__(self,
                 static_dim=9,
                 omni2_dim=57,
                 goes_dim=42,
                 d_model=128,
                 output_len=432,
                 nhead=4,
                 num_layers=2,
                 dropout=0.1):
        super().__init__()

        self.static_encoder = nn.Sequential(
            nn.Linear(static_dim, d_model),
            nn.ReLU(),
            nn.LayerNorm(d_model)
        )

        self.omni2_proj = nn.Linear(omni2_dim, d_model)
        self.goes_proj = nn.Linear(goes_dim, d_model)

        self.omni2_pos = PositionalEncoding(d_model)
        self.goes_pos = PositionalEncoding(d_model)

        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout, batch_first=True)
        self.omni2_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.goes_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fusion = nn.Sequential(
            nn.Linear(d_model * 3, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, output_len)
        )

    def forward(self, static_input, omni2_seq, goes_seq, omni2_mask=None, goes_mask=None):
        B = static_input.size(0)

        static_embed = self.static_encoder(static_input)

        omni2_embed = self.omni2_proj(omni2_seq)
        omni2_embed = self.omni2_pos(omni2_embed)
        omni2_key_mask = (~omni2_mask.bool()).any(dim=-1) if omni2_mask is not None else None
        omni2_out = self.omni2_encoder(omni2_embed, src_key_padding_mask=omni2_key_mask)
        omni2_summary = omni2_out.mean(dim=1)

        if goes_seq.shape[1] > 1024:
            step = goes_seq.shape[1] // 1024
            goes_seq = goes_seq[:, ::step, :]
            goes_mask = goes_mask[:, ::step, :] if goes_mask is not None else None

        goes_embed = self.goes_proj(goes_seq)
        goes_embed = self.goes_pos(goes_embed)
        goes_key_mask = (~goes_mask.bool()).any(dim=-1) if goes_mask is not None else None
        goes_out = self.goes_encoder(goes_embed, src_key_padding_mask=goes_key_mask)
        goes_summary = goes_out.mean(dim=1)

        combined = torch.cat((static_embed, omni2_summary, goes_summary), dim=-1)
        return self.fusion(combined)

def masked_mse_loss(preds, targets, mask, eps=1e-8):
    
    diff = (targets - preds) * mask
    sq = torch.square(diff)
    sum = torch.sum(sq)
    N = torch.sum(mask)
    # print(sum)
    # print(N)
    loss = torch.sqrt((sum/N))
    return loss


def eval_storm_transformer(initial_states_df, timestamps, device=None):

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    val_dataset = FullDataset(initial_states_df, timestamps)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

    model = STORMTransformer().to(device)
    model.load_state_dict(torch.load("../checkpoints/storm_epoch_7.pt", weights_only=False))

    ids_list = []
    timestamps_list = []
    preds_list = []

    counter = 0
    model.train()
    with torch.no_grad():
        for batch in tqdm(val_loader):
            # if counter >3:
            #     break
            counter += 1
            (static_input, 
            density_tensor, 
            density_mask_tensor, 
            goes_tensor, 
            goes_mask_tensor, 
            omni2_tensor, 
            omni2_mask_tensor, 
            timestamps,
            ids) = batch

            static_input = static_input.to(device)
            density_tensor = density_tensor.to(device)
            density_mask_tensor = density_mask_tensor.to(device)
            goes_tensor = goes_tensor.to(device)
            goes_mask_tensor = goes_mask_tensor.to(device)
            omni2_tensor = omni2_tensor.to(device)
            omni2_mask_tensor = omni2_mask_tensor.to(device)

            preds = model(static_input, omni2_tensor, goes_tensor, omni2_mask_tensor, goes_mask_tensor)
            #print (preds)
            loss = masked_mse_loss(preds,density_tensor,density_mask_tensor)

            ids_list = ids_list + ids.tolist()
            timestamps_list = timestamps_list + list(timestamps)
            preds_list = preds_list + (preds.to('cpu')).tolist()
            print ("Density tensor", density_tensor.to('cpu').tolist())
            print ("Preds: ", preds.to('cpu').tolist())
            print ("loss:",loss)
    # print (ids_list)
    # print (timestamps_list)
    # print (preds_list)
    
    pred_df = pd.DataFrame(preds_list)
    ids_df = pd.DataFrame(ids_list)
    timestamps_df = pd.DataFrame(timestamps_list)
    out = pd.concat([ids_df,timestamps_df, pred_df],axis=1)
    return (out)
            #break

In [8]:
class FullDatasetPT(Dataset):
    def __init__(self, initial_states_df, pt_dir='../data/processed/pt_files'):
        self.data = initial_states_df.reset_index(drop=True)
        self.pt_dir = pt_dir

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        file_id = str(int(row['File ID'])).zfill(5)
        pt_path = os.path.join(self.pt_dir, f"{file_id}.pt")

        if not os.path.exists(pt_path):
            raise FileNotFoundError(f".pt file not found for File ID: {file_id}")

        static_input = torch.tensor(row.drop("File ID").fillna(0.0).values, dtype=torch.float32)
        pt_data = torch.load(pt_path)

        return (
            static_input,
            pt_data["density"],
            pt_data["density_mask"],
            pt_data["goes"],
            pt_data["goes_mask"],
            pt_data["omni2"],
            pt_data["omni2_mask"]
        )



In [4]:
# -----------------------------------
# Positional Encoding for Sequences
# -----------------------------------
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=10000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(-torch.arange(0, d_model, 2) * math.log(10000.0) / d_model)
        pe[:, 0::2] = torch.sin(pos * div_term)
        pe[:, 1::2] = torch.cos(pos * div_term)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :].to(x.device)

# -----------------------------------
# STORMTransformer with Mask Handling
# -----------------------------------
class STORMTransformer(nn.Module):
    def __init__(self,
                 static_dim=9,
                 omni2_dim=57,
                 goes_dim=42,
                 d_model=128,
                 output_len=432,
                 nhead=4,
                 num_layers=2,
                 dropout=0.1):
        super().__init__()

        self.static_encoder = nn.Sequential(
            nn.Linear(static_dim, d_model),
            nn.ReLU(),
            nn.LayerNorm(d_model)
        )

        self.omni2_proj = nn.Linear(omni2_dim, d_model)
        self.goes_proj = nn.Linear(goes_dim, d_model)

        self.omni2_pos = PositionalEncoding(d_model)
        self.goes_pos = PositionalEncoding(d_model)

        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout, batch_first=True)
        self.omni2_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.goes_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fusion = nn.Sequential(
            nn.Linear(d_model * 3, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, output_len)
        )

    def forward(self, static_input, omni2_seq, goes_seq, omni2_mask=None, goes_mask=None):
        B = static_input.size(0)

        static_embed = self.static_encoder(static_input)

        omni2_embed = self.omni2_proj(omni2_seq)
        omni2_embed = self.omni2_pos(omni2_embed)
        omni2_key_mask = (~omni2_mask.bool()).any(dim=-1) if omni2_mask is not None else None
        omni2_out = self.omni2_encoder(omni2_embed, src_key_padding_mask=omni2_key_mask)
        omni2_summary = omni2_out.mean(dim=1)

        if goes_seq.shape[1] > 1024:
            step = goes_seq.shape[1] // 1024
            goes_seq = goes_seq[:, ::step, :]
            goes_mask = goes_mask[:, ::step, :] if goes_mask is not None else None
        global tester_mask
        global tester_seq
        tester_seq = goes_seq
        tester_mask = goes_mask
        #print(goes_seq,"\n",goes_mask)
        #print(goes_mask.sum())
        goes_embed = self.goes_proj(goes_seq)
        goes_embed = self.goes_pos(goes_embed)
        goes_key_mask = (~goes_mask.bool()).any(dim=-1) if goes_mask is not None else None
        global tester_key_mask
        global tester_embed
        tester_embed = goes_embed
        tester_key_mask = goes_key_mask
        goes_out = self.goes_encoder(goes_embed, src_key_padding_mask=goes_key_mask)
        goes_summary = goes_out.mean(dim=1)

        combined = torch.cat((static_embed, omni2_summary, goes_summary), dim=-1)
        return self.fusion(combined)

# -----------------------------------
# Masked MSE Loss
# -----------------------------------
def masked_mse_loss(preds, targets, mask, eps=1e-8):
    preds = torch.nan_to_num(preds, nan=0.0, posinf=1e3, neginf=0.0)
    targets = torch.nan_to_num(targets, nan=0.0, posinf=1e3, neginf=0.0)
    loss = (preds - targets) ** 2 * mask
    return loss.sum() / (mask.sum() + eps)

# -----------------------------------
# Full Training Loop with FullDataset
# -----------------------------------

        

In [15]:
def eval_storm_transformer(initial_states_df, num_epochs=10, batch_size=8, lr=1e-4, device=None):
    # from full_dataset import FullDataset
    # from storm_transformer import STORMTransformer, masked_mse_loss

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    os.makedirs("checkpoints", exist_ok=True)

    torch.manual_seed(42)

    # 🔀 Train/validation split
    #train_df, val_df = train_test_split(initial_states_df, test_size=0.05, random_state=42)

    #train_dataset = FullDatasetPT(train_df)
    val_dataset = FullDatasetPT(initial_states_df)

    # train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    # val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    #train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2, pin_memory=True, )
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=2, pin_memory=True)


    model = STORMTransformer().to(device)
    model.load_state_dict(torch.load("../checkpoints/storm_best.pt", weights_only=False))
    optimizer = optim.Adam(model.parameters(), lr=lr)

    checkpoint_path = "checkpoints/storm_last.pt"
    best_model_path = "checkpoints/storm_best.pt"
    start_epoch = 0
    best_val_loss = float("inf")

    # 🔁 Resume support
    # if os.path.exists(checkpoint_path):
    #     print(f"🔁 Resuming from checkpoint: {checkpoint_path}")
    #     checkpoint = torch.load(checkpoint_path, map_location=device)
    #     model.load_state_dict(checkpoint['model_state_dict'])
    #     optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    #     start_epoch = checkpoint.get('epoch', 0)
    #     best_val_loss = checkpoint.get('val_loss', float("inf"))

    # 🚀 Training loop
    
    for epoch in range(start_epoch, num_epochs):
        model.train()
        total_train_loss = 0.0

        # print(f"\n🚀 Epoch {epoch + 1}/{num_epochs}")
        # #start_load = time.time()
        # for batch in tqdm(train_loader):
        #     static_input, density, density_mask, goes, goes_mask, omni2, omni2_mask = batch
        #     #nd_load = time.time()

        #     static_input = static_input.to(device)
        #     density = density.to(device)
        #     density_mask = density_mask.to(device)
        #     goes = goes.to(device)
        #     goes_mask = goes_mask.to(device)
        #     omni2 = omni2.to(device)
        #     omni2_mask = omni2_mask.to(device)

        #     # if (omni2_mask.any(dim=-1).sum(dim=1) == 0).any() or (goes_mask.any(dim=-1).sum(dim=1) == 0).any():
        #     #     print("⚠️ Skipping batch with fully masked inputs")
        #     #     continue

        #     optimizer.zero_grad()
        #     preds = model(static_input, omni2, goes, omni2_mask, goes_mask)
        #     #print (preds)
        #     loss = masked_mse_loss(preds, density, density_mask)
        #     loss.backward()
        #     torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        #     optimizer.step()

        #     total_train_loss += loss.item()

            #end_batch = time.time()

            # print ("Load time:", end_load - start_load )
            # print ("Calc time:", end_batch - end_load)

        #avg_train_loss = total_train_loss / len(train_loader)

        # 🧪 Validation
        
        total_val_loss = 0.0
        with torch.no_grad():
            for batch_idx, batch in tqdm(enumerate(val_loader)):
                static_input, density, density_mask, goes, goes_mask, omni2, omni2_mask = batch

                static_input = static_input.to(device)
                density = density.to(device)
                density_mask = density_mask.to(device)
                goes = goes.to(device)
                goes_mask = goes_mask.to(device)
                omni2 = omni2.to(device)
                omni2_mask = omni2_mask.to(device)

                if (omni2_mask.any(dim=-1).sum(dim=1) == 0).any() or (goes_mask.any(dim=-1).sum(dim=1) == 0).any():
                    continue

                preds = model(static_input, omni2, goes, omni2_mask, goes_mask)
                loss = masked_mse_loss(preds, density, density_mask)
                total_val_loss += loss.item()

                # 🧠 Mask diagnostics
                goes_mask_sum = goes_mask.sum().item()
                omni2_mask_sum = omni2_mask.sum().item()
                density_mask_sum = density_mask.sum().item()
                print ("Preds:",preds)
                print ("DEnsity:",density)

                print(f"🧪 Eval Batch {batch_idx+1}/{len(val_loader)} — "
                      f"OMNI2 Mask Sum: {omni2_mask_sum} | "
                      f"GOES Mask Sum: {goes_mask_sum} | "
                      f"Density Mask Sum: {density_mask_sum}")

                # ⚠️ Alert if any mask has < 10% coverage
                if omni2_mask_sum < 0.1 * omni2_mask.numel():
                    print("⚠️ Low OMNI2 coverage in this batch!")
                if goes_mask_sum < 0.1 * goes_mask.numel():
                    print("⚠️ Low GOES coverage in this batch!")
                if density_mask_sum < 0.1 * density_mask.numel():
                    print("⚠️ Low density mask coverage in this batch!")

        break
        avg_val_loss = total_val_loss / len(val_loader)
        # print(f"\n📊 Epoch {epoch+1}/{num_epochs} — "
        #       f"Train Loss: {avg_train_loss:.20f} | Val Loss: {avg_val_loss:.20f}")
        print(f"\n📊 Epoch {epoch+1}/{num_epochs} — "
              f"Val Loss: {avg_val_loss:.20f}")

        # 💾 Save full checkpoint
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': avg_val_loss
        }, checkpoint_path)

        # 💎 Save best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), best_model_path)
            print("✅ Best model updated.")


In [7]:
# val_dataset = FullDataset(initial_states_normalized,timestamps)
# val_dataset[1]


In [16]:
temp = eval_storm_transformer(initial_states_normalized)

1it [00:06,  6.65s/it]

Preds: tensor([[1.1699e-12, 1.1677e-12, 1.0375e-12,  ..., 1.1656e-12, 1.0559e-12,
         1.2387e-12],
        [1.1699e-12, 1.1677e-12, 1.0375e-12,  ..., 1.1656e-12, 1.0559e-12,
         1.2387e-12],
        [1.1699e-12, 1.1677e-12, 1.0375e-12,  ..., 1.1656e-12, 1.0559e-12,
         1.2387e-12],
        ...,
        [1.1699e-12, 1.1677e-12, 1.0375e-12,  ..., 1.1656e-12, 1.0559e-12,
         1.2387e-12],
        [1.1699e-12, 1.1677e-12, 1.0375e-12,  ..., 1.1656e-12, 1.0559e-12,
         1.2387e-12],
        [1.1699e-12, 1.1677e-12, 1.0375e-12,  ..., 1.1656e-12, 1.0559e-12,
         1.2387e-12]], device='cuda:0')
DEnsity: tensor([[1.8036e-12, 1.8122e-12, 1.8238e-12,  ..., 1.5155e-12, 1.5238e-12,
         1.5268e-12],
        [1.5027e-12, 1.5005e-12, 1.5051e-12,  ..., 1.6731e-12, 1.6743e-12,
         1.6662e-12],
        [1.5366e-12, 1.5353e-12, 1.5500e-12,  ..., 1.5877e-12, 1.5848e-12,
         1.5878e-12],
        ...,
        [1.7939e-12, 1.7972e-12, 1.8041e-12,  ..., 0.0000e+00, 0.00

2it [00:08,  4.03s/it]

Preds: tensor([[1.1699e-12, 1.1677e-12, 1.0375e-12,  ..., 1.1656e-12, 1.0559e-12,
         1.2387e-12],
        [1.1699e-12, 1.1677e-12, 1.0375e-12,  ..., 1.1656e-12, 1.0559e-12,
         1.2387e-12],
        [1.1699e-12, 1.1677e-12, 1.0375e-12,  ..., 1.1656e-12, 1.0559e-12,
         1.2387e-12],
        ...,
        [1.1699e-12, 1.1677e-12, 1.0375e-12,  ..., 1.1656e-12, 1.0559e-12,
         1.2387e-12],
        [1.1699e-12, 1.1677e-12, 1.0375e-12,  ..., 1.1656e-12, 1.0559e-12,
         1.2387e-12],
        [1.1699e-12, 1.1677e-12, 1.0375e-12,  ..., 1.1656e-12, 1.0559e-12,
         1.2387e-12]], device='cuda:0')
DEnsity: tensor([[1.9602e-12, 1.9647e-12, 1.9593e-12,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [2.0488e-12, 2.0472e-12, 2.0476e-12,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.00

3it [00:10,  2.86s/it]

...

Preds: tensor([[1.1699e-12, 1.1677e-12, 1.0375e-12,  ..., 1.1656e-12, 1.0559e-12,
         1.2387e-12],
        [1.1699e-12, 1.1677e-12, 1.0375e-12,  ..., 1.1656e-12, 1.0559e-12,
         1.2387e-12],
        [1.1699e-12, 1.1677e-12, 1.0375e-12,  ..., 1.1656e-12, 1.0559e-12,
         1.2387e-12],
        ...,
        [1.1699e-12, 1.1677e-12, 1.0375e-12,  ..., 1.1656e-12, 1.0559e-12,
         1.2387e-12],
        [1.1699e-12, 1.1677e-12, 1.0375e-12,  ..., 1.1656e-12, 1.0559e-12,
         1.2387e-12],
        [1.1699e-12, 1.1677e-12, 1.0375e-12,  ..., 1.1656e-12, 1.0559e-12,
         1.2387e-12]], device='cuda:0')
DEnsity: tensor([[2.8777e-12, 2.8820e-12, 2.9016e-12,  ..., 2.1418e-12, 2.1430e-12,
         2.1717e-12],
        [2.3123e-12, 2.2836e-12, 2.3084e-12,  ..., 2.1380e-12, 2.1385e-12,
         2.1537e-12],
        [2.1421e-12, 2.1412e-12, 2.1691e-12,  ..., 2.1223e-12, 2.1203e-12,
         2.1239e-12],
        ...,
        [2.0746e-12, 2.0680e-12, 2.0511e-12,  ..., 1.4928e-12, 1.48

24it [00:41,  1.49s/it]

Preds: tensor([[1.1699e-12, 1.1677e-12, 1.0375e-12,  ..., 1.1656e-12, 1.0559e-12,
         1.2387e-12],
        [1.1699e-12, 1.1677e-12, 1.0375e-12,  ..., 1.1656e-12, 1.0559e-12,
         1.2387e-12],
        [1.1699e-12, 1.1677e-12, 1.0375e-12,  ..., 1.1656e-12, 1.0559e-12,
         1.2387e-12],
        ...,
        [1.1699e-12, 1.1677e-12, 1.0375e-12,  ..., 1.1656e-12, 1.0559e-12,
         1.2387e-12],
        [1.1699e-12, 1.1677e-12, 1.0375e-12,  ..., 1.1656e-12, 1.0559e-12,
         1.2387e-12],
        [1.1699e-12, 1.1677e-12, 1.0375e-12,  ..., 1.1656e-12, 1.0559e-12,
         1.2387e-12]], device='cuda:0')
DEnsity: tensor([[1.5689e-12, 1.5744e-12, 1.5741e-12,  ..., 1.5865e-12, 1.5747e-12,
         1.5784e-12],
        [1.6572e-12, 1.6446e-12, 1.6332e-12,  ..., 1.3643e-12, 1.3501e-12,
         1.3330e-12],
        [1.4728e-12, 1.4648e-12, 1.4585e-12,  ..., 1.3065e-12, 1.2969e-12,
         1.2920e-12],
        ...,
        [1.0511e-12, 1.0575e-12, 1.0630e-12,  ..., 1.5001e-12, 1.49

24it [00:42,  1.75s/it]


KeyboardInterrupt: 

In [7]:
temp

,0,0,0,1,2,3,4,5,6,7,...,422,423,424,425,426,427,428,429,430,431
0,0,2000-08-02 04:50:33,2.032928e-12,-0.000002,1.304894e-07,-0.000008,3.175190e-09,2.600965e-09,1.468148e-10,1.770449e-08,...,1.131555e-11,4.252406e-10,2.851178e-08,-1.048421e-10,-6.560407e-10,-2.387168e-07,4.361981e-09,7.714585e-08,-8.475395e-09,5.980621e-08
1,1,2000-08-03 19:51:01,2.032928e-12,-0.000002,1.304894e-07,-0.000008,3.175190e-09,2.600965e-09,1.468148e-10,1.770449e-08,...,1.131555e-11,4.252406e-10,2.851178e-08,-1.048421e-10,-6.560407e-10,-2.387168e-07,4.361981e-09,7.714585e-08,-8.475395e-09,5.980621e-08
2,2,2000-08-05 05:40:05,2.032928e-12,-0.000002,1.304894e-07,-0.000008,3.175190e-09,2.600965e-09,1.468148e-10,1.770449e-08,...,1.131555e-11,4.252406e-10,2.851178e-08,-1.048421e-10,-6.560407e-10,-2.387168e-07,4.361981e-09,7.714585e-08,-8.475395e-09,5.980621e-08
3,3,2000-08-06 05:02:20,2.032928e-12,-0.000002,1.304894e-07,-0.000008,3.175190e-09,2.600965e-09,1.468148e-10,1.770449e-08,...,1.131555e-11,4.252406e-10,2.851178e-08,-1.048421e-10,-6.560407e-10,-2.387168e-07,4.361981e-09,7.714585e-08,-8.475395e-09,5.980621e-08
4,4,2000-08-08 20:54:57,2.032928e-12,-0.000002,1.304894e-07,-0.000008,3.175190e-09,2.600965e-09,1.468148e-10,1.770449e-08,...,1.131555e-11,4.252406e-10,2.851178e-08,-1.048421e-10,-6.560407e-10,-2.387168e-07,4.361981e-09,7.714585e-08,-8.475395e-09,5.980621e-08
5,5,2000-08-09 20:17:05,2.032928e-12,-0.000002,1.304894e-07,-0.000008,3.175190e-09,2.600965e-09,1.468148e-10,1.770449e-08,...,1.131555e-11,4.252406e-10,2.851178e-08,-1.048421e-10,-6.560407e-10,-2.387168e-07,4.361981e-09,7.714585e-08,-8.475395e-09,5.980621e-08
6,6,2000-08-11 19:01:16,2.032928e-12,-0.000002,1.304894e-07,-0.000008,3.175190e-09,2.600965e-09,1.468148e-10,1.770449e-08,...,1.131555e-11,4.252406e-10,2.851178e-08,-1.048421e-10,-6.560407e-10,-2.387168e-07,4.361981e-09,7.714585e-08,-8.475395e-09,5.980621e-08
7,7,2000-09-03 02:50:45,2.032928e-12,-0.000002,1.304894e-07,-0.000008,3.175190e-09,2.600965e-09,1.468148e-10,1.770449e-08,...,1.131555e-11,4.252406e-10,2.851178e-08,-1.048421e-10,-6.560407e-10,-2.387168e-07,4.361981e-09,7.714585e-08,-8.475395e-09,5.980621e-08
8,8,2000-09-05 15:40:48,2.032928e-12,-0.000002,1.304894e-07,-0.000008,3.175190e-09,2.600965e-09,1.468148e-10,1.770449e-08,...,1.131555e-11,4.252406e-10,2.851178e-08,-1.048421e-10,-6.560407e-10,-2.387168e-07,4.361981e-09,7.714585e-08,-8.475395e-09,5.980621e-08
9,9,2000-09-06 02:35:56,2.032928e-12,-0.000002,1.304894e-07,-0.000008,3.175190e-09,2.600965e-09,1.468148e-10,1.770449e-08,...,1.131555e-11,4.252406e-10,2.851178e-08,-1.048421e-10,-6.560407e-10,-2.387168e-07,4.361981e-09,7.714585e-08,-8.475395e-09,5.980621e-08
